In [1]:
# 필요한 라이브러리 import

import platform
import folium
import math
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

if platform.system() == 'Darwin': # 맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': # 윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': # 리눅스 (구글 Colab)
        plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings(action = 'ignore')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# 데이터셋 다운로드
from geoband.API import *

GetCompasData('SBJ_2102_003', '3', '3.대전광역시_신호등(보행등).geojson')
GetCompasData('SBJ_2102_003', '4', '4.대전광역시_신호등(차량등).geojson')
GetCompasData('SBJ_2102_003', '10', '10.대전광역시_교통CCTV.geojson')
GetCompasData('SBJ_2102_003', '16', '16.대전광역시_기상데이터(2017~2019).csv')
GetCompasData('SBJ_2102_003', '19', '19.대전광역시_상세도로망(2018).geojson')
GetCompasData('SBJ_2102_003', '20', '20.대전광역시_평일_일별_시간대별_추정교통량(2018).csv')
GetCompasData('SBJ_2102_003', '21', '21.대전광역시_평일_일별_혼잡빈도강도(2018).csv')
GetCompasData('SBJ_2102_003', '22', '22.대전광역시_평일_일별_혼잡시간강도(2018).csv')

daejeon_signal_walk = gpd.read_file('3.대전광역시_신호등(보행등).geojson')
daejeon_signal_car = gpd.read_file('4.대전광역시_신호등(차량등).geojson')
cctv = gpd.read_file('10.대전광역시_교통CCTV.geojson')
weather = pd.read_csv('16.대전광역시_기상데이터(2017~2019).csv')
road_detail = gpd.read_file('19.대전광역시_상세도로망(2018).geojson')
day_time_accident = pd.read_csv('20.대전광역시_평일_일별_시간대별_추정교통량(2018).csv')
day_time_confuse = pd.read_csv('21.대전광역시_평일_일별_혼잡빈도강도(2018).csv')
day_time_times = pd.read_csv('22.대전광역시_평일_일별_혼잡시간강도(2018).csv')

## 5. accident_count_filter 데이터 불러오기(의미가 떨어지는 사고건수가 0인 데이터 Filter)

In [3]:
accident_gid = pd.read_csv('accident_gid.csv', index_col = 0)
print(np.shape(accident_gid))
accident_gid.head(3)

(25097, 4)


,gid,acci_cnt,geometry_2,geometry
0,다바866110,0.0,LINESTRING (127.3486909175812 36.2983602119806...,MULTIPOLYGON (((127.3507618774412 36.296222652...
1,다바823157,0.0,LINESTRING (127.3039387031694 36.3388912702199...,MULTIPOLYGON (((127.3027655376963 36.338525724...
2,다바823157,0.0,LINESTRING (127.3039387031694 36.3388912702199...,MULTIPOLYGON (((127.3027655376963 36.338525724...


In [4]:
accident_count_filter = pd.read_csv('accident_count_filter.csv')
accident_count_filter['geometry'] = [wkt.loads(line) for line in list(accident_count_filter['geometry'])]
print(np.shape(accident_count_filter))
accident_count_filter.head(3)

(5556, 7)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042


## 6. 교통안전시설물 - 보행자 신호등의 geometry에 따라 그에 일치하는 gid(격자) Labeling 및 병합

In [5]:
def get_gid(criteria, df) : # within 함수 사용
    gids = pd.DataFrame()
    gidss = []
    for i in tqdm(list(df.index)) :
        for j in list(criteria.index) : 
            if df.loc[i, 'geometry'].within(criteria.loc[j, 'geometry']) : # criteria와 df의 geometry가 교차하는지 확인
                gids = pd.concat([gids, df.loc[[i,]]])
                gidss.append(criteria.loc[j, 'gid'])
                break
    gids['gid'] = gidss
    return gids

get_gid 함수를 통해 **criteria와 df의 geometry 값에 대하여 두 데이터가 서로 교차(within)하는지 확인하여 그에 맞는 gid를 Labeling**한다.<br/>
이 함수를 통하여 gid 격자가 없는 데이터들을 Labeling 해서 1번 과정에서 만들었던 5,556개 격자 데이터와 병합하는 작업을 수행하도록 한다.

In [6]:
## 보행자 신호등
signal_walk_samp = get_gid(accident_count_filter, daejeon_signal_walk)
signal_walk_samp.to_csv('final_signal_walk.csv')
signal_walk_samp.head()

In [7]:
final_signal_walk = pd.read_csv('final_signal_walk.csv', index_col = 0)
print(np.shape(final_signal_walk))
final_signal_walk.head()

(5552, 8)


,gu,dong,jibun,loc_cd,sgnl_drn_cd,sgnl_knd_cd,geometry,gid
0,서구,관저동,NaN,1.0,NaN,2,POINT (127.334619467568 36.29501431648465),다바851108
1,유성구,노은동,NaN,1.0,NaN,2,POINT (127.3235205239782 36.3670329930303),다바841188
2,동구,판암동,NaN,1.0,NaN,2,POINT (127.4613672442361 36.32029083293295),다바965136
7,동구,천동,NaN,1.0,NaN,2,POINT (127.4445737985495 36.31713656664272),다바950133
8,동구,천동,NaN,1.0,NaN,2,POINT (127.4444815869447 36.31659667794215),다바950132


In [8]:
signal_walk_count = final_signal_walk.groupby('gid').count()[['geometry']]
signal_walk_count.reset_index(inplace=True)
signal_walk_count.columns = ['gid', '신호등_보행자수']

accident_count_filter = accident_count_filter.merge(signal_walk_count, on = 'gid', how = 'left')
accident_count_filter['신호등_보행자수'] = accident_count_filter['신호등_보행자수'].fillna(0)
print(np.shape(accident_count_filter))
accident_count_filter.head(3)

(5556, 8)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0


## 7. 교통안전시설물 - 차량 신호등의 geometry에 따라 그에 일치하는 gid(격자) Labeling 및 병합

In [9]:
## 차량 신호등
signal_car_samp = get_gid(accident_count_filter, daejeon_signal_car)
signal_car_samp.to_csv('final_signal_car.csv')
signal_car_samp.head()

In [10]:
final_signal_car = pd.read_csv('final_signal_car.csv', index_col = 0)
print(np.shape(final_signal_car))
final_signal_car.head()

(8794, 8)


,gu,dong,jibun,loc_cd,sgnl_drn_cd,sgnl_knd_cd,geometry,gid
0,유성구,구암동,NaN,1,2,1,POINT (127.329215606638 36.34866636469033),다바846168
1,유성구,구암동,NaN,1,2,1,POINT (127.3292166841326 36.34861926175856),다바846168
2,유성구,구암동,NaN,1,2,1,POINT (127.3293927765914 36.34835870534383),다바846167
3,동구,삼정동,11-17전,1,2,1,POINT (127.4776798809445 36.33255601008285),다바979150
4,유성구,구암동,NaN,1,2,1,POINT (127.3295454232338 36.34861674022679),다바847168


In [11]:
signal_car_count = final_signal_car.groupby('gid').count()[['geometry']]
signal_car_count.reset_index(inplace=True)
signal_car_count.columns = ['gid', '신호등_차량등수']

accident_count_filter = accident_count_filter.merge(signal_car_count, on = 'gid', how = 'left')
accident_count_filter['신호등_차량등수'] = accident_count_filter['신호등_차량등수'].fillna(0)
print(np.shape(accident_count_filter))
accident_count_filter.head(3)

(5556, 9)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0,3.0
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0,0.0
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0,0.0


## 8. 교통안전시설물 - CCTV의 geometry에 따라 그에 일치하는 gid(격자) Labeling 및 병합

In [12]:
## CCTV
cctv_samp = get_gid(accident_count_filter, cctv)
cctv_samp.to_csv('final_cctv.csv')
cctv_samp

In [13]:
final_cctv = pd.read_csv('final_cctv.csv', index_col = 0)
print(np.shape(final_cctv))
final_cctv.head()

(118, 5)


,gu,dong,jibun,geometry,gid
0,대덕구,오정동,NaN,POINT (127.4152345319938 36.35875724574742),다바923179
1,대덕구,오정동,NaN,POINT (127.4152392844907 36.35876507827574),다바923179
2,서구,탄방동,NaN,POINT (127.401849404806 36.34503114747884),다바911164
3,동구,가양동,NaN,POINT (127.4423500116377 36.35097538486666),다바948170
4,서구,용문동,NaN,POINT (127.393257603758 36.33811979940576),다바904156


In [14]:
cctv_count = final_cctv.groupby('gid').count()[['geometry']]
cctv_count.reset_index(inplace=True)
cctv_count.columns = ['gid', 'cctv수']

accident_count_filter = accident_count_filter.merge(cctv_count, on = 'gid', how = 'left')
accident_count_filter['cctv수'] = accident_count_filter['cctv수'].fillna(0)
print(np.shape(accident_count_filter))
accident_count_filter.head(3)

(5556, 10)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0,3.0,0.0
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0,0.0,0.0
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0,0.0,0.0


In [15]:
accident_count_filter.to_csv('accident_count_filter_1.csv') # 신호등(보행등), 신호등(차량등), cctv수까지 merge한 상태.

## 9. 교통혼잡빈도 데이터의 geometry에 따라 그에 일치하는 gid(격자) Labeling 및 병합

In [16]:
accident_count_filter_1 = pd.read_csv('accident_count_filter_1.csv', index_col = 0)
accident_count_filter_1['geometry'] = [wkt.loads(line) for line in list(accident_count_filter_1['geometry'])] # str 형태에서 geometry 형태로 형 변환
print(np.shape(accident_count_filter_1))
accident_count_filter_1.head(3)

(5556, 10)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0,3.0,0.0
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0,0.0,0.0
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0,0.0,0.0


In [17]:
day_time_confuse['link_id'] = [int(id/100) for id in list(day_time_confuse['상세도로망_LinkID'])] # 상세도로망_LinkID로 link_id 추출 
road_detail = road_detail.astype({'link_id' : 'int64'})
day_time_confused = day_time_confuse.merge(road_detail, on='link_id', how='left') # 위에서 만든 link_id를 gid가 있는 road_detail 데이터와 병합
print(np.shape(day_time_confused))
day_time_confused.head(2)

(18044, 27)


,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡빈도강도,link_id,max_speed,...,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,up_its_id,dw_its_id,geometry
0,56421933101,101,0.121,경부고속도로,대전광역시,대덕구,덕암동,28.24,564219331,100,...,0,1,0.121,3,1,0,0,1870196300,0,"MULTILINESTRING ((127.41780 36.43107, 127.4181..."
1,56421763601,101,0.244,경부고속도로,대전광역시,대덕구,덕암동,32.06,564217636,100,...,0,1,0.244,3,1,0,0,1870196000,0,"MULTILINESTRING ((127.41688 36.42635, 127.4168..."


In [18]:
def get_gid2(criteria, df) : # 도로 형태의 Polygon 형태. 따라서 within이 아닌 crosses 사용
    gids = pd.DataFrame()
    gidss = []
    for i in tqdm(list(df.index)) :
        for j in list(criteria.index) : 
            if df.loc[i, 'geometry'].crosses(criteria.loc[j, 'geometry']) : 
                gids = pd.concat([gids, df.loc[[i,]]])
                gidss.append(criteria.loc[j, 'gid'])
                break
    gids['gid'] = gidss
    return gids

In [19]:
## 교통혼잡도
accident_count_filter_2 = get_gid2(accident_count_filter_1, day_time_confused)
accident_count_filter_2.to_csv('accident_count_filter_2.csv')
accident_count_filter_2

In [20]:
accident_count_filter_2 = pd.read_csv('accident_count_filter_2.csv', index_col = 0)
accident_count_filter_2['geometry'] = [wkt.loads(line) for line in list(accident_count_filter_2['geometry'])]
print(np.shape(accident_count_filter_2))
accident_count_filter_2.head(3)

(10028, 28)


,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡빈도강도,link_id,max_speed,...,oneway,length,width,car_lane,num_cross,barrier,up_its_id,dw_its_id,geometry,gid
1,56421763601,101,0.244,경부고속도로,대전광역시,대덕구,덕암동,32.06,564217636,100,...,1,0.244,3,1,0,0,1870196000,0,(LINESTRING (127.4168847073474 36.426348006591...,다바925253
7,56420113501,101,1.151,경부고속도로,대전광역시,대덕구,덕암동,36.20,564201135,100,...,1,1.151,3,1,0,0,1870198000,0,(LINESTRING (127.4172644168076 36.453428662051...,다바927278
8,56420113201,101,1.151,경부고속도로,대전광역시,대덕구,덕암동,42.36,564201132,100,...,1,1.151,3,1,0,0,1870198100,0,(LINESTRING (127.4197828255578 36.443253549970...,다바927278


In [21]:
confused = accident_count_filter_2.groupby('gid').mean()[['혼잡빈도강도']] # 격자별로 혼잡빈도강도의 평균 값을 사용
confused.reset_index(inplace=True)

accident_count_filter_1 = accident_count_filter_1.merge(confused, on = 'gid', how = 'left')
accident_count_filter_1['혼잡빈도강도'] = accident_count_filter_1['혼잡빈도강도'].fillna(0)
print(np.shape(accident_count_filter_1))
accident_count_filter_1.head(3)

(5556, 11)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수,혼잡빈도강도
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0,3.0,0.0,97.36
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0,0.0,0.0,62.85
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0,0.0,0.0,94.56


In [22]:
accident_count_filter_1.to_csv('accident_count_filter_3.csv') # 신호등(보행등), 신호등(차량등), cctv수, 혼잡빈도까지 merge한 상태.

## 10. 교통혼잡시간 데이터의 geometry에 따라 그에 일치하는 gid(격자) Labeling 및 병합

In [23]:
accident_count_filter_3 = pd.read_csv('accident_count_filter_3.csv', index_col = 0)
accident_count_filter_3['geometry'] = [wkt.loads(line) for line in list(accident_count_filter_3['geometry'])]
print(np.shape(accident_count_filter_3))
accident_count_filter_3.head(3)

(5556, 11)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수,혼잡빈도강도
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0,3.0,0.0,97.36
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0,0.0,0.0,62.85
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0,0.0,0.0,94.56


In [24]:
day_time_times['link_id'] = [int(id/100) for id in list(day_time_confuse['상세도로망_LinkID']) ]
day_time_timed = day_time_times.merge(road_detail, on='link_id', how='left')
print(np.shape(day_time_timed))
day_time_timed.head(2)

(18044, 27)


,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡시간강도,link_id,max_speed,...,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,up_its_id,dw_its_id,geometry
0,56421933101,101,0.121,경부고속도로,대전광역시,대덕구,덕암동,35.50,564219331,100,...,0,1,0.121,3,1,0,0,1870196300,0,"MULTILINESTRING ((127.41780 36.43107, 127.4181..."
1,56421763601,101,0.244,경부고속도로,대전광역시,대덕구,덕암동,47.61,564217636,100,...,0,1,0.244,3,1,0,0,1870196000,0,"MULTILINESTRING ((127.41688 36.42635, 127.4168..."


In [25]:
## 교통혼잡시간
accident_count_filter_4 = get_gid2(accident_count_filter_3, day_time_timed)
accident_count_filter_4.to_csv('accident_count_filter_4.csv')
accident_count_filter_4

In [26]:
accident_count_filter_4 = pd.read_csv('accident_count_filter_4.csv', index_col = 0)
accident_count_filter_4 = accident_count_filter_4.reset_index(drop = True)
accident_count_filter_4['geometry'] = [wkt.loads(line) for line in list(accident_count_filter_4['geometry'])]
print(np.shape(accident_count_filter_4))
accident_count_filter_4.head(3)

(10028, 28)


,상세도로망_LinkID,도로등급,링크길이,도로명,시도명,시군구명,읍면동명,혼잡시간강도,link_id,max_speed,...,oneway,length,width,car_lane,num_cross,barrier,up_its_id,dw_its_id,geometry,gid
0,56421763601,101,0.244,경부고속도로,대전광역시,대덕구,덕암동,47.61,564217636,100,...,1,0.244,3,1,0,0,1870196000,0,(LINESTRING (127.4168847073474 36.426348006591...,다바925253
1,56420113501,101,1.151,경부고속도로,대전광역시,대덕구,덕암동,43.17,564201135,100,...,1,1.151,3,1,0,0,1870198000,0,(LINESTRING (127.4172644168076 36.453428662051...,다바927278
2,56420113201,101,1.151,경부고속도로,대전광역시,대덕구,덕암동,47.59,564201132,100,...,1,1.151,3,1,0,0,1870198100,0,(LINESTRING (127.4197828255578 36.443253549970...,다바927278


In [27]:
timed = accident_count_filter_4.groupby('gid').mean()[['혼잡시간강도']] # 격자별로 혼잡시간강도의 평균 값을 사용
timed.reset_index(inplace=True)

accident_count_filter_5 = accident_count_filter_3.merge(timed, on = 'gid', how = 'left')
accident_count_filter_5['혼잡시간강도'] = accident_count_filter_5['혼잡시간강도'].fillna(0)
print(np.shape(accident_count_filter_5))
accident_count_filter_5.head(3)

(5556, 12)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수,혼잡빈도강도,혼잡시간강도
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0,3.0,0.0,97.36,98.5000
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0,0.0,0.0,62.85,86.1475
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0,0.0,0.0,94.56,17.4900


## 11. 시간대별 추정교통량 데이터의 geometry에 따라 그에 일치하는 gid(격자) Labeling 및 병합

In [28]:
day_time_accident['link_id'] = [int(id/100) for id in list(day_time_accident['상세도로망_LinkID']) ]
day_time_accidentd = day_time_accident.merge(road_detail, on='link_id', how='left')
day_time_accidentd.groupby('link_id').mean('전체_추정교통량')
day_time_accidentd.reset_index(inplace=True)

In [29]:
day_time_accidentd = day_time_accidentd.merge(accident_count_filter_2[['gid', 'link_id']], on='link_id', how='left')
day_time_sample = day_time_accidentd.loc[:, ['전체_추정교통량', 'gid']]
day_time_sample = day_time_sample.groupby('gid').mean()
day_time_sample.reset_index(inplace = True)

In [30]:
accident_count_filter_5 = accident_count_filter_5.merge(day_time_sample, on = 'gid', how = 'left')
accident_count_filter_5 = accident_count_filter_5.fillna(0)
print(np.shape(accident_count_filter_5))
accident_count_filter_5.head(5)

(5556, 13)


,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,cctv수,혼잡빈도강도,혼잡시간강도,전체_추정교통량
0,다바931203,2,(POLYGON ((127.4230710131166 36.38013455218083...,2,2,127.423628,36.380586,1.0,3.0,0.0,97.360000,98.500000,927.740000
1,다바861174,0,(POLYGON ((127.3450791441312 36.35391426501025...,0,0,127.345636,36.354366,0.0,0.0,0.0,62.850000,86.147500,280.790000
2,다바900127,1,(POLYGON ((127.3886063974092 36.31159021601022...,1,1,127.389163,36.312042,0.0,0.0,0.0,94.560000,17.490000,568.760000
3,다바885170,15,(POLYGON ((127.3718339133395 36.35033979440872...,15,15,127.372390,36.350791,0.0,0.0,0.0,38.888333,54.821667,1710.090000
4,다바957151,4,"(POLYGON ((127.452087475365 36.33326960850717,...",4,4,127.452644,36.333721,2.0,3.0,0.0,62.542222,75.783333,185.816471


In [31]:
accident_count_filter_5.to_csv('accident_count_filter_5.csv') # 신호등(보행등), 신호등(차량등), cctv수, 혼잡빈도, 혼잡시간, 교통추정량까지 merge한 상태.

## weather 변수를 연도별로 나눠주기

In [32]:
weather_2017 = weather[weather['일시'] < '2018-01-01']
weather_2018 = weather[(weather['일시'] >= '2018-01-01') & (weather['일시'] <= '2018-12-31')].reset_index(drop = True)
weather_2019 = weather[weather['일시'] >= '2019-01-01'].reset_index(drop = True)

In [33]:
weather_2017.to_csv('weather_2017.csv')
weather_2018.to_csv('weather_2018.csv')
weather_2019.to_csv('weather_2019.csv')